# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction import text
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings("ignore")

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [7]:
m = Mystem()
texts =[]
for text in data['text'].values:
    text = text.lower()
    token_text = w_tokenizer.tokenize(text)
    lemm_text = lemmatizer.lemmatize(str(token_text))
    clear_text = re.sub(r'[^a-zA-Z]', ' ' , str(lemm_text))
    text_clear = " ".join(clear_text.split())
    texts.append(text_clear)
    
lemmatized_text = pd.Series(texts, name='lemmatized_text')
data_lemmatized = pd.concat([data, lemmatized_text], axis=1)
data_lemmatized

,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,and for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,you should be ashamed of yourself that is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,spitzer umm theres no actual article for prost...
159569,And it looks like it was actually you who put ...,0,and it looks like it was actually you who put ...


In [8]:
data_lemmatized_features, data_lemmatized_test = train_test_split(data_lemmatized, test_size = 0.2, random_state=12345)
target_test = data_lemmatized_test['toxic']
features_test = data_lemmatized_test.drop(['toxic'], axis = 1)
data_lemmatized_features

,text,toxic,lemmatized_text
45800,I have offered as well...but he must do my map...,0,i have offered as well but he must do my map f...
94209,YOUR biased! ORTHODOX VIEWS have no more merit...,0,your biased orthodox views have no more merit ...
135210,Thank you very much for your quick respond and...,0,thank you very much for your quick respond and...
89158,"I moved this page. When I did, I found anothe...",0,i moved this page when i did i found another l...
61233,Gray Powell article nominated for deletion \nN...,0,gray powell article nominated for deletion nom...
...,...,...,...
109993,"Gam, keep your CRAP off my Talk Page\n\nYour K...",0,gam keep your crap off my talk page your kkk s...
85412,I corrected what you listed.-,0,i corrected what you listed
133249,"""\n\nhmm, yes, as I said: I also """"watch"""" the...",0,hmm yes as i said i also watch the emily ruete...
130333,kinda like miachael jackson,0,kinda like miachael jackson


In [9]:
features_test

,text,lemmatized_text
146790,Ahh shut the fuck up you douchebag sand nigger...,ahh shut the fuck up you douchebag sand nigger...
2941,"""\n\nREPLY: There is no such thing as Texas Co...",reply there is no such thing as texas commerce...
115087,"Reply\nHey, you could at least mention Jasenov...",reply hey you could at least mention jasenovac...
48830,"Thats fine, there is no deadline ) chi?",thats fine there is no deadline chi
136034,"""\n\nDYK nomination of Mustarabim\n Hello! You...",dyk nomination of mustarabim hello your submis...
...,...,...
32895,"I agree with Paul, and i'm going to revert it ...",i agree with paul and i m going to revert it w...
18757,I never came across a rule that you can't use ...,i never came across a rule that you can t use ...
5679,Autzen was not built on a landfill! The landfi...,autzen was not built on a landfill the landfil...
139386,"""IF YOU DONT LIKE ME LEAVING NASTY MESSAGES AN...",if you dont like me leaving nasty messages and...


In [10]:
count_tf_idf = TfidfVectorizer(stop_words = 'english', ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1, 
                               smooth_idf=1, sublinear_tf=1)
features_train = count_tf_idf.fit_transform(data_lemmatized_features['lemmatized_text'].values)
target_train = data_lemmatized_features['toxic']
features_test = count_tf_idf.transform(features_test['lemmatized_text'].values)
X = data_lemmatized_features['lemmatized_text']
X_test = data_lemmatized_test['lemmatized_text']

Загружен файл с размеченными токсичными комментариями. Файл состоит из 159 тыс. записей. Файл не содержит пропуски в данных. 10 % (несбалансированные данные) комментариев отмеченны как токсичные. Лемметизируем столбец text для его последующей векторизации и использовании в моделях машинного обучения. Выделим тренировочную и тестовую выборки целевых показателей и признаков.

## Обучение

### Логистическая регрессия

In [11]:
score = make_scorer(f1_score, greater_is_better=True)

In [12]:
param_set = {
 'lr__max_iter': [100]
}

In [13]:
pipe_lr = Pipeline([('vect', TfidfVectorizer(stop_words = 'english', ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1, 
                             smooth_idf=1, sublinear_tf=1)),
                    ('tfidf', TfidfTransformer()),
                    ('lr', LogisticRegression(random_state = 12345, class_weight='balanced'))
                   ])


In [14]:
model_lr = GridSearchCV(estimator = pipe_lr, param_grid=param_set, cv=2, scoring = score)
model_lr.fit(X, target_train)
f1_score_lr = model_lr.score(X, target_train)
print(f'Мера f1 для логистической регрессии:{f1_score_lr}')

Мера f1 для логистической регрессии:0.9051796743402584


### Случайный лес

In [15]:
pipe_rf = Pipeline([('vect', TfidfVectorizer(stop_words = 'english', ngram_range=(1,3), min_df=3, max_df=0.9, use_idf=1, 
                             smooth_idf=1, sublinear_tf=1)),
                    ('tfidf', TfidfTransformer()),
                    ('rf', RandomForestClassifier(random_state = 12345, class_weight='balanced'))
                   ])

In [16]:
param_set = {
 'rf__n_estimators':[100, 120, 150, 200],
 'rf__max_depth': [12, 15, 20, 25]
}

In [17]:
model_rf = GridSearchCV(estimator = pipe_rf, param_grid = param_set, cv=2, scoring = score)
model_rf.fit(X, target_train)
f1_score_rf = model_rf.score(X, target_train)
print(f'Мера f1 для случайного леса:{f1_score_rf}')

Мера f1 для случайного леса:0.38152494549185584


In [18]:
model_rf.best_params_

{'rf__max_depth': 15, 'rf__n_estimators': 150}

Обучены модели логистической регрессии без подбора параметров и случайного леса с подбором параметров, в гиперпараметрах обеих моделей указан параметр балансировки несбалансированных данных. F1 мера для логистической регрессии 0,76, для случайного леса 0,39.

## Выводы

### Логистическая регрессия

In [19]:
predict_lr = model_lr.predict(X_test)
f1_score_lr_test = f1_score(target_test, predict_lr)
print(f'Мера f1 для логистической регрессии:{f1_score_lr_test}')

Мера f1 для логистической регрессии:0.7696563774024462


In [20]:
print(classification_report(target_test, predict_lr, 
                            target_names=['not_toxic', 'toxic']))

              precision    recall  f1-score   support

   not_toxic       0.98      0.97      0.97     28676
       toxic       0.73      0.82      0.77      3239

    accuracy                           0.95     31915
   macro avg       0.85      0.89      0.87     31915
weighted avg       0.95      0.95      0.95     31915



### Случайный лес

In [21]:
predict_rf = model_rf.predict(X_test)
f1_score_rf_test = f1_score(target_test, predict_rf)
print(f'Мера f1 для случайного леса:{f1_score_rf_test}')

Мера f1 для случайного леса:0.36432418568863806


In [22]:
print(classification_report(target_test, predict_rf, 
                            target_names=['not_toxic', 'toxic']))

              precision    recall  f1-score   support

   not_toxic       0.98      0.67      0.79     28676
       toxic       0.23      0.88      0.36      3239

    accuracy                           0.69     31915
   macro avg       0.60      0.77      0.58     31915
weighted avg       0.90      0.69      0.75     31915



### Случайный классификатор

In [23]:
dc = DummyClassifier(random_state = 12345, strategy='uniform')
dc.fit(X, target_train)
predict_dc = dc.predict(X_test)
f1_score_dc = f1_score(target_test, predict_dc)
print(f'Мера f1 для мнимый классификатор :{f1_score_dc}')

Мера f1 для мнимый классификатор :0.16850426461410445


In [24]:
print(classification_report(target_test, predict_dc, 
                            target_names=['not_toxic', 'toxic']))

              precision    recall  f1-score   support

   not_toxic       0.90      0.50      0.64     28676
       toxic       0.10      0.50      0.17      3239

    accuracy                           0.50     31915
   macro avg       0.50      0.50      0.41     31915
weighted avg       0.82      0.50      0.59     31915



На тестовой выборке сделаны предсказания токсичности комментариев и рассчитана f1 мера. Логистическая регрессия - 0,76, случайный лес - 0,37, мнимый классификатор - 0.17. Для предсказаний классификации текстовых данных предлагается использовать модель логистической регрессии.